In [0]:
import tensorflow as tf
from tensorflow.keras import layers
from PIL import Image
import os
import pathlib
import numpy as np
import pathlib
import shutil

In [0]:
DATA_PATH = '/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Resized_Data'
# train_dir = pathlib.Path(DATA_PATH+'/training')
# val_dir = pathlib.Path(DATA_PATH+'/val')

# train_dir = dbutils.fs.ls(DATA_PATH+"/train/")
# val_dir = dbutils.fs.ls(DATA_PATH+"/val/")
batch_size = 128
img_height = 120
img_width = 160

resized_height = 120
resized_width = 160

# IMG_SHAPE = (224, 224, 3)
CLASS_PEOPLE = "/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/training/people_class"
num_classes = 2
NUM_EPOCHS = 120
seed = 12
log_dir = "/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/logs"

STEPS_PER_EPOCH = 1508 //batch_size

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
data_augmentation = tf.keras.Sequential([
#   layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.2),
  layers.experimental.preprocessing.RandomZoom(0.2),
  layers.experimental.preprocessing.RandomContrast(0.2),
#   layers.experimental.preprocessing.RandomContrast(0.2),
])

resize = tf.keras.layers.experimental.preprocessing.Resizing(resized_height, resized_width)

def preprocess_input(image):
    return (image/127.5) - 1

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATA_PATH,
  seed=seed,
  shuffle=True,
  validation_split=0.2,
  subset='training',
  image_size=(img_height, img_width)
)
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

train_ds = train_ds.map(
  lambda x, y: (preprocess_input(x), y))


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATA_PATH,
  seed=seed,
  shuffle=True,
  validation_split=0.2,
  subset='validation',
  image_size=(img_height, img_width)
)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(
  lambda x, y: (preprocess_input(x), y))

Found 7910 files belonging to 2 classes.
Using 6328 files for training.
Found 7910 files belonging to 2 classes.
Using 1582 files for validation.

In [0]:
list(val_ds)

Out[4]: [(<tf.Tensor: shape=(32, 120, 160, 3), dtype=float32, numpy=
 array([[[[ 0.10588241, 0.27843142, 0.11372554],
 [ 0.12156868, 0.2941177 , 0.12941182],
 [ 0.12941182, 0.30196083, 0.13725495],
 ...,
 [-0.54509807, -0.41176468, -0.56078434],
 [-0.5372549 , -0.42745095, -0.62352943],
 [-0.54509807, -0.44313723, -0.64705884]],
 
 [[ 0.09019613, 0.26274514, 0.09803927],
 [ 0.10588241, 0.27843142, 0.11372554],
 [ 0.12941182, 0.30196083, 0.13725495],
 ...,
 [-0.5294118 , -0.38823527, -0.56078434],
 [-0.5137255 , -0.41176468, -0.6156863 ],
 [-0.5294118 , -0.42745095, -0.64705884]],
 
 [[ 0.06666672, 0.23921573, 0.07450986],
 [ 0.09019613, 0.26274514, 0.09803927],
 [ 0.12156868, 0.2941177 , 0.12941182],
 ...,
 [-0.4980392 , -0.372549 , -0.5764706 ],
 [-0.49019605, -0.41176468, -0.62352943],
 [-0.5058824 , -0.41960782, -0.654902 ]],
 
 ...,
 
 [[-0.5137255 , -0.42745095, -0.56078434],
 [-0.5058824 , -0.41960782, -0.5529412 ],
 [-0.5058824 , -0.41960782, -0.5529412 ],
 ...,
 [-0.5058824 , -0.3960784 , -0.5764706 ],
 [-0.5058824 , -0.3960784 , -0.5764706 ],
 [-0.5058824 , -0.3960784 , -0.5764706 ]],
 
 [[-0.5137255 , -0.42745095, -0.56078434],
 [-0.5058824 , -0.41960782, -0.5529412 ],
 [-0.5058824 , -0.41960782, -0.5529412 ],
 ...,
 [-0.5137255 , -0.40392154, -0.58431375],
 [-0.5137255 , -0.40392154, -0.58431375],
 [-0.5058824 , -0.3960784 , -0.5764706 ]],
 
 [[-0.5137255 , -0.42745095, -0.56078434],
 [-0.5058824 , -0.41960782, -0.5529412 ],
 [-0.5058824 , -0.41960782, -0.5529412 ],
 ...,
 [-0.5137255 , -0.40392154, -0.58431375],
 [-0.5137255 , -0.40392154, -0.58431375],
 [-0.5137255 , -0.40392154, -0.58431375]]],
 
 
 [[[-0.44313723, -0.31764704, -0.40392154],
 [-0.44313723, -0.31764704, -0.40392154],
 [-0.4352941 , -0.3098039 , -0.3960784 ],
 ...,
 [-0.372549 , -0.26274508, -0.36470586],
 [-0.372549 , -0.26274508, -0.36470586],
 [-0.36470586, -0.25490195, -0.35686272]],
 
 [[-0.45098037, -0.32549018, -0.41176468],
 [-0.45098037, -0.32549018, -0.41176468],
 [-0.44313723, -0.31764704, -0.40392154],
 ...,
 [-0.38039213, -0.27058822, -0.372549 ],
 [-0.3960784 , -0.2862745 , -0.38823527],
 [-0.3960784 , -0.2862745 , -0.38823527]],
 
 [[-0.46666664, -0.34117645, -0.42745095],
 [-0.4588235 , -0.3333333 , -0.41960782],
 [-0.45098037, -0.32549018, -0.41176468],
 ...,
 [-0.3960784 , -0.2862745 , -0.38823527],
 [-0.41960782, -0.3098039 , -0.41176468],
 [-0.42745095, -0.31764704, -0.41960782]],
 
 ...,
 
 [[-0.6392157 , -0.4980392 , -0.60784316],
 [-0.62352943, -0.4823529 , -0.5921569 ],
 [-0.6 , -0.47450978, -0.5764706 ],
 ...,
 [-0.5137255 , -0.42745095, -0.47450978],
 [-0.52156866, -0.45098037, -0.49019605],
 [-0.5372549 , -0.45098037, -0.4980392 ]],
 
 [[-0.6313726 , -0.49019605, -0.6 ],
 [-0.62352943, -0.4823529 , -0.5921569 ],
 [-0.5921569 , -0.46666664, -0.5686275 ],
 ...,
 [-0.4352941 , -0.36470586, -0.40392154],
 [-0.4352941 , -0.38823527, -0.41960782],
 [-0.4588235 , -0.38823527, -0.42745095]],
 
 [[-0.6313726 , -0.49019605, -0.6 ],
 [-0.6156863 , -0.47450978, -0.58431375],
 [-0.5921569 , -0.46666664, -0.5686275 ],
 ...,
 [-0.35686272, -0.3098039 , -0.34117645],
 [-0.35686272, -0.3098039 , -0.34117645],
 [-0.35686272, -0.3098039 , -0.34117645]]],
 
 
 [[[-0.75686276, -0.5764706 , -0.6392157 ],
 [-0.7647059 , -0.58431375, -0.64705884],
 [-0.77254903, -0.5921569 , -0.654902 ],
 ...,
 [-0.9372549 , -0.8117647 , -0.8980392 ],
 [-0.94509804, -0.81960785, -0.90588236],
 [-0.94509804, -0.81960785, -0.90588236]],
 
 [[-0.73333335, -0.5529412 , -0.6156863 ],
 [-0.7411765 , -0.56078434, -0.62352943],
 [-0.7490196 , -0.5686275 , -0.6313726 ],
 ...,
 [-0.92156863, -0.79607844, -0.88235295],
 [-0.92941177, -0.8039216 , -0.8901961 ],
 [-0.92941177, -0.8039216 , -0.8901961 ]],
 
 [[-0.69411767, -0.5137255 , -0.5764706 ],
 [-0.70980394, -0.5294118 , -0.5921569 ],
 [-0.7254902 , -0.54509807, -0.60784316],
 ...,
 [-0.92156863, -0.79607844, -0.88235295],
 [-0.92941177, -0.8039216 , -0.8901961 ],
 [-0.92941177, -0.8039216 , -0.8901961 ]],
 
 ...,
 
 [[

In [0]:
def get_model(lr=0.001, alpha=0.35):
  base_model = tf.keras.applications.MobileNetV2(
      input_shape=(img_height, img_width, 3),
      include_top=False, weights='imagenet', alpha=alpha)
  #     base_model.trainable = False
  base_model.trainable = True
  x = base_model.output
  # Add a new classifier layer for transfer learning
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
  #     prediction_layer = tf.keras.layers.Dense(1, activation='softmax')
  model = tf.keras.Model(inputs=base_model.inputs, outputs=x)
  return model

def get_compiled_model(alpha=0.35, lr=0.001):
  model = get_model(lr, alpha)
  #     lr = learning_rate = CustomSchedule(2)
  model.compile(
              optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9),
  #                 optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  return model

In [0]:
def train_and_evaluate(train_ds, val_ds, lr=0.001):
    model = get_compiled_model()
#     model = get_compiled_model_2()
    checkpoint_path = "/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/Weight2/version-{epoch:04d}.h5py"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    
    model.save_weights(checkpoint_path.format(epoch=0))

    # 创建一个保存模型权重的回调
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)
#                                                      save_freq=5)
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10)
    board_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=0,  # How often to log histogram visualizations
        embeddings_freq=0,  # How often to log embedding visualizations
        update_freq="epoch",
    )  # How often to write logs (default: once per epoch)
    
#     steps_per_epoch = len(train_ds) // batch_size
    hist = model.fit(train_ds, 
#                      steps_per_epoch=steps_per_epoch,
                     epochs=NUM_EPOCHS,
                     validation_data=val_ds,
#                      class_weight={0:0.3, 1:0.7},
#                      validation_steps=validation_steps,
                     verbose=2,
                     callbacks=[
                       cp_callback, 
                       board_callback, 
                       early_stop,
#                        CustomLearningRateScheduler(lr_schedule),
                     ])
#     model.save('saved_model/my_model')
    return hist,model

In [0]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
      print('yes')
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
yes
yes
2 Physical GPUs, 2 Logical GPUs

In [0]:
tf.debugging.set_log_device_placement(True)
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  hist,model = train_and_evaluate(train_ds, val_ds)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
WARNING:tensorflow:`input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_0.35_224_no_top.h5
 8192/2019640 [..............................] - ETA: 0s2023424/2019640 [==============================] - 0s 0us/step
Epoch 1/120
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow/python/data/ops/multi_device_iterator_ops.py:601: get_next_as_optional (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0781s vs `on_train_batch_end` time: 0.3532s). Check your callbacks.

Epoch 00001: saving model to /dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/Weight2/version-0001.h5py
198/198 - 83s - loss: 0.5804 - accuracy: 0.7107 - val_loss: 0.5672 - val_accuracy: 0.7295
Epoch 2/120

Epoch 00002: saving model to /dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/Weight2/version-0002.h5py
198/198 - 14s - loss: 0.4869 - accuracy: 0.8189 - val_loss: 0.4890 - val_accuracy: 0.8217
Epoch 3/120

Epoch 00003: saving model to /dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/Weight2/version-0003.h5py
198/198 - 14s - loss: 0.4443 - accuracy: 0.8663 - val_loss: 0.4551 - val_accuracy: 0.8565
Epoch 4/120

Epoch 00004: saving model to /dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/Weight2/version-0004.h5py
198/198 - 14s - loss: 0.4265 - accuracy: 0.8842 - val_loss: 0.4366 - val_accuracy: 0.8717
Epoch 5/120

Epoch 00005: saving model to /dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/Weight2/ve

In [0]:
def load_model(checkpoint_path):
  model = get_compiled_model()
  # load weights
  model.load_weights(checkpoint_path)
  return model
  
def predict(img_path, model):
  test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    img_path,
#     seed=seed,
    shuffle=False,
#     validation_split=0.2,
#     subset='validation',
    image_size=(img_height, img_width),
    batch_size=batch_size)

#   test_ds_2 = test_ds.map(
#   lambda x, y: (resize(x), y))
  test_ds_2 = test_ds.map(
    lambda x, y: (preprocess_input(x), y))
  
#     with open(CLASS_PEOPLE + '.pkl','rb') as f:
#         class_to_index = pickle.load(f)

  result = model.predict(test_ds_2, batch_size=batch_size)
  scores = []
  for i in range(len(result)):
      scores.append(result[i][-1])
  print(scores)
  return result, test_ds, scores
#   for item in result:
#       print(np.argmax(item))

In [0]:
model = load_model('/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/Weight2/version-0072.h5py')
model.summary()

WARNING:tensorflow:`input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type) Output Shape Param # Connected to 
==================================================================================================
input_3 (InputLayer) [(None, 120, 160, 3) 0 
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D) (None, 121, 161, 3) 0 input_3[0][0] 
__________________________________________________________________________________________________
Conv1 (Conv2D) (None, 60, 80, 16) 432 Conv1_pad[0][0] 
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization) (None, 60, 80, 16) 64 Conv1[0][0] 
__________________________________________________________________________________________________
Conv1_relu (ReLU) (None, 60, 80, 16) 0 bn_Conv1[0][0] 
__________________________________________________________________________________________________
expanded_conv_depthwise (Depthw (None, 60, 80, 16) 144 Conv1_relu[0][0] 
__________________________________________________________________________________________________
expanded_conv_depthwise_BN (Bat (None, 60, 80, 16) 64 expanded_conv_depthwise[0][0] 
__________________________________________________________________________________________________
expanded_conv_depthwise_relu (R (None, 60, 80, 16) 0 expanded_conv_depthwise_BN[0][0] 
__________________________________________________________________________________________________
expanded_conv_project (Conv2D) (None, 60, 80, 8) 128 expanded_conv_depthwise_relu[0][0
__________________________________________________________________________________________________
expanded_conv_project_BN (Batch (None, 60, 80, 8) 32 expanded_conv_project[0][0] 
__________________________________________________________________________________________________
block_1_expand (Conv2D) (None, 60, 80, 48) 384 expanded_conv_project_BN[0][0] 
__________________________________________________________________________________________________
block_1_expand_BN (BatchNormali (None, 60, 80, 48) 192 block_1_expand[0][0] 
__________________________________________________________________________________________________
block_1_expand_relu (ReLU) (None, 60, 80, 48) 0 block_1_expand_BN[0][0] 
__________________________________________________________________________________________________
block_1_pad (ZeroPadding2D) (None, 61, 81, 48) 0 block_1_expand_relu[0][0] 
__________________________________________________________________________________________________
block_1_depthwise (DepthwiseCon (None, 30, 40, 48) 432 block_1_pad[0][0] 
__________________________________________________________________________________________________
block_1_depthwise_BN (BatchNorm (None, 30, 40, 48) 192 block_1_depthwise[0][0] 
__________________________________________________________________________________________________
block_1_depthwise_relu (ReLU) (None, 30, 40, 48) 0 block_1_depthwise_BN[0][0] 
__________________________________________________________________________________________________
block_1_project (Conv2D) (None, 30, 40, 8) 384 block_1_depthwise_relu[0][0] 
__________________________________________________________________________________________________
block_1_project_BN (BatchNormal (None, 30, 40, 8) 32 block_1_project[0][0] 
__________________________________________________________________________________________________
block_2_expand (Conv2D) (None, 30, 40, 48) 384 block_1_project_BN[0][0] 
__________________________________________________________________________________________________
block_2_expand_BN (BatchNormali (None, 30, 40, 48) 192 block_2_expand[0][0] 
_________________________________________

In [0]:
model = tf.keras.models.load_model('/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/best_model')

WARNING:tensorflow:Unresolved object in checkpoint: (root).optimizer.iter
WARNING:tensorflow:Unresolved object in checkpoint: (root).optimizer.decay
WARNING:tensorflow:Unresolved object in checkpoint: (root).optimizer.learning_rate
WARNING:tensorflow:Unresolved object in checkpoint: (root).optimizer.momentum
WARNING:tensorflow:A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.
Exception ignored in: 'h5py._objects.ObjectID.__dealloc__'
Traceback (most recent call last):
 File "h5py/_objects.pyx", line 193, in h5py._objects.ObjectID.__dealloc__
RuntimeError: Can't decrement id ref count (file write failed: time = Thu Mar 18 09:52:16 2021
, filename = '/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/best_model.h5', file descriptor = 33, errno = 95, error message = 'Operation not supported', buf = 0x562a47935d40, total write size = 1728, bytes this sub-write = 1728, bytes actually written = 18446744073709551615, offset = 95040)

--------------------------------------------------------------------------- 
 RuntimeError Traceback (most recent call last)
 h5py/_objects.pyx in h5py._objects.ObjectID.__dealloc__ () 

 RuntimeError : Can't decrement id ref count (file write failed: time = Thu Mar 18 09:52:16 2021
, filename = '/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/best_model.h5', file descriptor = 33, errno = 95, error message = 'Operation not supported', buf = 0x562a47935d40, total write size = 1728, bytes this sub-write = 1728, bytes actually written = 18446744073709551615, offset = 95040) --------------------------------------------------------------------------- 
 OSError Traceback (most recent call last)
 /databricks/python/lib/python3.7/site-packages/tensorflow/python/keras/saving/hdf5_format.py in save_model_to_hdf5 (model, filepath, overwrite, include_optimizer) 
 118 model_layers = model . layers
 --> 119 save_weights_to_hdf5_group ( model_weights_group , model_layers ) 
 120 

 /databricks/python/lib/python3.7/site-packages/tensorflow/python/keras/saving/hdf5_format.py in save_weights_to_hdf5_group (f, layers) 
 644 else : 
 --> 645 param_dset [ : ] = val
 646 

 h5py/_objects.pyx in h5py._objects.with_phil.wrapper () 

 h5py/_objects.pyx in h5py._objects.with_phil.wrapper () 

 /databricks/python/lib/python3.7/site-packages/h5py/_hl/dataset.py in __setitem__ (self, args, val) 
 707 for fspace in selection . broadcast ( mshape ) : 
 --> 708 self . id . write ( mspace , fspace , val , mtype , dxpl = self . _dxpl ) 
 709 

 h5py/_objects.pyx in h5py._objects.with_phil.wrapper () 

 h5py/_objects.pyx in h5py._objects.with_phil.wrapper () 

 h5py/h5d.pyx in h5py.h5d.DatasetID.write () 

 h5py/_proxy.pyx in h5py._proxy.dset_rw () 

 h5py/_proxy.pyx in h5py._proxy.H5PY_H5Dwrite () 

 OSError : Can't write data (file write failed: time = Thu Mar 18 09:52:16 2021
, filename = '/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/best_model.h5', file descriptor = 33, errno = 95, error message = 'Operation not supported', buf = 0x562a457ba190, total write size = 573440, bytes this sub-write = 573440, bytes actually written = 18446744073709551615, offset = 101216)

During handling of the above exception, another exception occurred:

 RuntimeError Traceback (most recent call last)
 <command-243523631348296> in <module> 
 1 model = tf . keras . models . load_model ( '/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/best_model' ) 
 ----> 2 model . save ( '/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model/best_model.h5' ) 

 /databricks/python/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py in save (self, filepath, overwrite, include_optimizer, save_format, signatures, options) 
 1977 """
 1978 save.save_model(self, filepath, overwrite, include_optimizer, save_format,
 -> 1979 signatures, options)
 1980 
 1981 def save_weights(self,

 /databricks/python/lib/python3.7/site-packages/tensorflow/python/keras/saving/save.py in save_model (model, filepath, overwrite, include_optimizer, save_format, signatures, options) 
 129 'or using `save_weights`.')
 130 hdf5_format.save_model_to_hdf5(
 --> 131 model, filepath, overwrite, include_optimizer)
 132 else : 
 133 saved_model_save.save(model, filepath, overwrite, include_optimizer,

 /databricks/python/lib/python3.7/site-packages/tensorflow/python/keras/saving/hdf5_format.py in save_model_to_hdf5 (model, filepath, overwrite, include_optimizer) 
 128 finally : 
 129 if opened_new_file : 
 --> 130 f . close ( ) 
 131 
 132 

 /databricks/python/lib/python3.7/site-packages/h5py/_hl/files.py in close (self) 
 441 for id_ in file_list : 
 442 while id_ . valid : 
 --> 443 h5i . dec_ref ( id_ ) 
 444 
 445 self . id . close ( ) 

 h5py/_objects.pyx in h5py._objects.with_phil.wrapper () 

 h5py/_objects.pyx in h5py._objects.with_phil.wrapper () 

 h5py/h5i.pyx in h5py.h5i.dec_ref () 

 RuntimeError : Dirty entry flush destroy failed (file write failed: time = Thu Mar 18 0

In [0]:
predict = model.predict(val_ds)
predict

Out[33]: array([[1.0352677e-08, 1.0000000e+00],
 [4.4936731e-11, 1.0000000e+00],
 [9.9997604e-01, 2.3958348e-05],
 ...,
 [9.9999988e-01, 1.2724522e-07],
 [2.1866257e-05, 9.9997818e-01],
 [1.0000000e+00, 3.9633860e-10]], dtype=float32)

In [0]:
predict.shape

Out[34]: (1582, 2)

In [0]:
y_true = []
y = []
val_list = list(val_ds)
for length in range(len(val_list)):
  y_true.append(list(np.array(val_list[length][1])))
for i in y_true:
  for t in i:
    y.append(t)
y

Out[35]: [1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,

In [0]:
y_predict = []
for i in predict:
  i = i.tolist()
  y_predict.append(i.index(max(i)))
y_predict

Out[36]: [1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

ytest = np.array(y)
y_pred = np.array(y_predict)
print('Accuracy score: ', accuracy_score(ytest,y_pred))
print('Precision score: ',precision_score(ytest,y_pred,average='macro'))
print('Recall: ', recall_score(ytest,y_pred,average='macro'))
print('F1 score: ',f1_score(ytest,y_pred,average='macro'))

Accuracy score: 0.9816687737041719
Precision score: 0.9816697466420268
Recall: 0.9816304459120462
F1 score: 0.9816497028346205

In [0]:
base_path = '/dbfs/FileStore/shared_uploads/lilian.lu@otis.com/'

img_path = DATA_PATH
# img_path = '/dbfs/FileStore/shared_uploads/lilian.lu@otis.com/images/11_26'
model_path = base_path+"training/cp-0066.ckpt"
# CLASS_PEOPLE = 'checkpoint/people_class'

model = load_model(model_path)
result, test_ds, scores = predict(img_path, model)

In [0]:
y_p = []
for item in result:
  y_p.append(np.argmax(item))

y_r = []
for x, y in test_ds:
    y_n = y.numpy()
    y_r.extend(y_n)

In [0]:
m_p = tf.keras.metrics.Precision()
m_r = tf.keras.metrics.Recall()
# m_auc = tf.keras.metrics.AUC()

m_p.update_state(y_r, y_p)
m_r.update_state(y_r, y_p)
# m_auc.update_state(y_r, y_p)

precision = m_p.result().numpy()
recall = m_r.result().numpy()
# auc = m_auc().result.numpy()
print("precision:%s\nrecall:%s"%(precision, recall))